<a href="https://colab.research.google.com/github/readall/indiatranslator/blob/main/indicTranslate_Transliter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%bash
rm -rf sample_data
rm -rf fairseq

In [4]:
%%bash
rm -rf sample_data
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers easyocr
pip install mosestokenizer indic-nlp-library mock sacrebleu tensorboardX pyarrow subword-nmt
git clone https://github.com/pytorch/fairseq.git
cd fairseq
pip install --editable ./
%cd ..
# pip install fairseq


In [5]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 498, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 498 (delta 160), reused 132 (delta 117), pack-reused 297
Receiving objects: 100% (498/498), 1.49 MiB | 10.08 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 12.51 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 31.

In [6]:
%%bash
# downloading the indic-indic model
wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
unzip indic-en.zip
rm indic-en.zip
wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
unzip m2m.zip
rm m2m.zip

# %cd indicTrans

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/preprocess.log  
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/test.SRC-TGT.SRC.idx  
  inflating: indic-en/final_bin/test.SRC-TGT.TGT.idx  
  inflating: indic-en/final_bin/train.SRC-TGT.SRC.idx  
  inflating: indic-en/final_bin/dict.SRC.txt  
  inflating: indic-en/final_bin/valid.SRC-TGT.TGT.idx  
  inflating: indic-en/final_bin/test.SRC-TGT.TGT.bin  
  inflating: indic-en/final_bin/valid.SRC-TGT.TGT.bin  
  inflating: indic-en/final_bin/train.SRC-TGT.TGT.idx  
  inflating: indic-en/final_bin/train.SRC-TGT.TGT.bin  
  inflating: indic-en/final_bin/valid.SRC-TGT.SRC.idx  
  inflating: indic-en/final_bin/train.SRC-TGT.SRC.bin  
  inflating: ind

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
!pwd
!cp /content/m2m/vocab/bpe_codes.32k.SRC_TGT /content/m2m/vocab/bpe_codes.32k.SRC


/content/indicTrans


In [6]:
%%bash
apt-get install python-numpy libicu-dev
pip install pyicu pycld2 morfessor 
pip install polyglot


Reading package lists...
Building dependency tree...
Reading state information...
python-numpy is already the newest version (1:1.13.3-2ubuntu1).
python-numpy set to manually installed.
libicu-dev is already the newest version (60.2-3ubuntu3.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
  Created wheel for pyicu: filename=PyICU-2.7.4-cp37-cp37m-linux_x86_64.whl size=1374693 sha256=f3f904ffe0f8629c0964d85f71599443f0f2b8b149de5e95f714679ecc20644b
  Stored in directory: /root/.cache/pip/wheels/c4/3a/c8/2d2397cd8f6cd04489ecac3d1e6821bb4938d73312d69b834f
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834148 sha256=9ffd5d7afaa66860a40435604f239d1998c658303e29f1a6c6f27156ae017ee2
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built

In [7]:
!polyglot download embeddings2.en transliteration2.ar
!polyglot download transliteration2.kn

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package transliteration2.ar to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package transliteration2.kn to
[polyglot_data]     /root/polyglot_data...


In [ ]:
# !rm indic-en.zip m2m.zip

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import easyocr

In [ ]:
reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
image_root_path = "/content/drive/MyDrive/Colab Notebooks/"
kannada_img_file = image_root_path + 'kannada_01.jpg'
kannada_img_file_1 = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'

In [ ]:
result = reader.readtext(kannada_img_file, detail = 0)
with open('ka.txt', 'w') as f:
    f.write(str(result))

result

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


['|.ಹೊಂದಿಸಿ ಬರೆಯಿರಿ:',
 '೧.ಕಾಪಾಡು',
 'ಹನಿ',
 '೨',
 'ಬುದ್ದಿವಂತ',
 'ಸಂರಕ್ಷಿಸು',
 '೩.ಮಳೆ',
 'ಆಸನ',
 '೪',
 'ಬೆಟ್ಟ',
 'ರಾಮಕೃಷ್ಣ',
 '೫ ಪೀಠ',
 'ಗುಡ್ಡ',
 'Iಆವರಣದಲ್ಲಿ ಕೊಟ್ಟಿರುವ ಸರಿಯಾದ ಉತ್ತರವನ್ನು ಆರಿಸಿ ಬಿಟ್ಟ ಸ್ಥಳದಲ್ಲಿ ಬರೆಯಿರಿ:',
 '(ಕುವೆಂಪು, ಅಪ್ಪುಗೆ, ಪಂಡಿತರು,',
 'ತ,ಮಳೆ',
 '೧',
 '.ಕೃಷ್ಣದೇವರಾಯನ ಆಸ್ಥಾನದಲ್ಲಿ',
 'ಇದ್ದರು.',
 '೨.ನೀರಿನ ಮೂಲ',
 'ಸಂಸ್ಕೃ ತ']

In [ ]:
# from transformers import AutoTokenizer, AutoModel
  
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", keep_accents=True)

# model = AutoModel.from_pretrained("ai4bharat/indic-bert")

In [8]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# !bash ./indicTrans/joint_translate.sh ka.txt en_outputs.txt 'kn' 'en' './indic-en'

In [1]:
# we need to be in indicTrans directory for translation model to load
%cd /content/indicTrans/


/content/indicTrans


In [ ]:
# !bash joint_translate.sh /content/ka.txt en_outputs.txt 'kn' 'en' '../indic-en'

In [ ]:
# !cat en_outputs.txt

In [ ]:
# from indicTrans.inference.engine import Model

# indic2en_model = Model(expdir='/content/indic-en')
# indic2en_model.batch_translate(result, 'kn', 'en')

In [ ]:
!pwd

/content/indicTrans


In [ ]:
# !cd ..

In [ ]:
!cd 
!cd /content/indicTrans

In [2]:
from indicTrans.inference.engine import Model
# indicm2m_model = Model(expdir='/content/m2m')
# indicm2m_model.batch_translate(result, 'kn', 'hi')

In [16]:
def KannadaImageToHindiText(kannada_img_file):
  import torch
  import numpy as np
  import matplotlib.pyplot as plt  
  import easyocr
  from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
  from indicTrans.inference.engine import Model

  reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
  result = reader.readtext(kannada_img_file, detail = 0)
  indicm2m_model = Model(expdir='/content/m2m')
  return indicm2m_model.batch_translate(result, 'kn', 'hi')


In [8]:
def cleanTransliterText(transliter_text):
  blank = ['',"", " ", ' ', '\t']
  s_list = []
  for x in transliter_text:
    if x not in blank:
      s_list.append(x)
  return s_list


In [3]:
def getTextFromImage(image_file):
  import torch
  import numpy as np
  import easyocr
  reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
  result = reader.readtext(image_file, detail = 0)
  return result

In [14]:
# KannadaImageToHindiText(kannada_img_file_1)
%cd
%cd /content/indicTrans/
%pwd

/root
/content/indicTrans


'/content/indicTrans'

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt  
import easyocr
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from indicTrans.inference.engine import Model

class KannadaToHindi :
  def __init__(self):
    self.reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    self.indicm2m_model = Model(expdir='/content/m2m')

  def TranslateKannadaImageToHindi(self, kannada_img_file):
    result = self.reader.readtext(kannada_img_file, detail = 0)
    return self.indicm2m_model.batch_translate(result, 'kn', 'hi')

In [5]:
traslator = KannadaToHindi()

Initializing vocab and bpe
Initializing model for translation


In [ ]:
kannada_img_file_1 = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'
traslator.TranslateKannadaImageToHindi(kannada_img_file_1)

In [ ]:
kannada_img_file_2 = '/content/drive/MyDrive/Colab Notebooks/kannada_02.jpg'
traslator.TranslateKannadaImageToHindi(kannada_img_file_2)

In [ ]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_03.jpg'
traslator.TranslateKannadaImageToHindi(kannada_img_file)

In [ ]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_04.jpg'
traslator.TranslateKannadaImageToHindi(kannada_img_file)

In [ ]:
from polyglot.downloader import downloader
# print(downloader.supported_languages_table("transliteration2"))

  1. Italian                    2. Hindi                      3. French                   
  4. Spanish; Castilian         5. Vietnamese                 6. Arabic                   
  7. Macedonian                 8. Bosnian-Croatian-Serbian   9. Norwegian Nynorsk        
 10. Azerbaijani               11. Bulgarian                 12. Georgian                 
 13. Galician                  14. Amharic                   15. Yiddish                  
 16. Norwegian                 17. Estonian                  18. Japanese                 
 19. Haitian; Haitian Creole   20. Belarusian                21. Greek, Modern            
 22. Welsh                     23. Albanian                  24. Marathi (Marāṭhī)        
 25. Armenian                  26. Slovene                   27. Korean                   
 28. Irish                     29. Bengali                   30. Serbian                  
 31. Finnish                   32. Catalan; Valencian        33. Croatian                 

In [10]:
from polyglot.text import Text

kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_04.jpg'
image_text = str(getTextFromImage( kannada_img_file))
text = Text(image_text)
transliter_text = text.transliterate("en")
trans_str = " ".join(cleanTransliterText(transliter_text))
print(trans_str)
# for x in text.transliterate("en"):
#   print(x)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


prashneglige erdu muru wakyadlli vidasagrn mukh arlitu ramkrishna vidasagrnondige wad madlu karna vidasagrn mukh arlitu mhishbndn endrenu endre elelin kddiglu mhishbndhn endre eamme kttuv hgg ide mhishbndhnd arth kottiruv matnnu aru arige helidru visha modle tilididdre nanu illige bruttle matnnu vidasagr krishnadevraanige helidnu idu nammurin dnkaauv hudugrigu gottu matnnu tenali ramkrishna vidasagrnige helidnu kelgin kvigl pricha madi kuvenpu purna hesru kuppleli venktpp puttpp jnn sthl shivmogg jillea kuppleli irlill


In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt  
import easyocr
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from indicTrans.inference.engine import Model
from polyglot.text import Text

class KannadaToHindi :
  def __init__(self):
    self.reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    self.indicm2m_model = Model(expdir='/content/m2m')


  def getTextFromImage(self, image_file):
    reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    result = reader.readtext(image_file, detail = 0)
    return result

  def cleanTransliterText(self, transliter_text):
    blank = ['',"", " ", ' ', '\t']
    s_list = []
    for x in transliter_text:
      if x not in blank:
        s_list.append(x)
    return s_list

  def getTransliterredText(self,text_from_image):
    image_text = str(text_from_image)
    text = Text(image_text)
    transliter_text = text.transliterate("en")
    trans_str = " ".join(cleanTransliterText(transliter_text))
    return trans_str

  def TranslateKannadaImageToHindi(self, kannada_img_file):
    text_from_image = self.reader.readtext(kannada_img_file, detail = 0)
    transliterred_result = self.getTransliterredText(text_from_image)
    translated_result = self.indicm2m_model.batch_translate(text_from_image, 'kn', 'hi')
    return translated_result, transliterred_result


In [14]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_04.jpg'
transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

Initializing vocab and bpe
Initializing model for translation


100%|██████████| 28/28 [00:00<00:00, 9326.60it/s]
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


(['नीचे दिए गए प्रश्नों के लिए दो/तीन पंक्ति में', 'विद्यासागर का चेहरा क्यों फूला?', 'उथेनाली रामकृष्ण विद्यासागर से बहस करने के लिए सहमत हुई', 'विद्यासागर का चेहरा फूट पड़ा', 'तिलकस्थ महिषाबंदन क्या है?', '0:', 'तिलकाष्ठा', '"" "" "" "बैल की छड़ियां" "" "और" "" "महिषबंधन" "" "यानि बैल की रस्सी" ""', 'तिलकत्था महिषाबंधन का अर्थ', 'ग्यारहवीं का वादा किसने किया?', 'यह विषय पहले से है', 'अगर मैं जानता हूं तो यहां आऊंगा।', 'जवाबः इस', 'विद्यासागर ने कृष्णदेवराय से कहा था -', 'हमारे पशुपालक भी यह जानते हैं।', 'उत्तरः यह', 'तेनाली रामकृष्ण ने विद्यासागर से कहा था -', 'बारहवें कवियों का परिचय देंः', '[शानदार]', 'पूरा नाम: कुप्पली वेंकटप्पा पुट्टप्पा', 'पैदा हुआ।', '29', '12', '1904', 'स्थान (P)', 'शिवमोगा जिले के कुपली', 'जवाब दीजिएः', 'नहीं था।'], 'prashneglige erdu muru wakyadlli vidasagrn mukh arlitu ramkrishna vidasagrnondige wad madlu karna vidasagrn mukh arlitu mhishbndn endrenu endre elelin kddiglu mhishbndhn endre eamme kttuv hgg ide mhishbndhnd arth kottiruv matnnu aru arige helidru

In [16]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_03.jpg'
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 16/16 [00:00<00:00, 8628.04it/s]


wakyadlli bisil niru niru aviaaitu mod awag bharwaaitu avi tnidu bharwaaitu vidasagrnondige wad madlu aru vidasagrnondige wad madlu tenali ramkrishna tenali ramkrishna wadkke av grnthvnnu arisikondnu ramkrishna wadkke mhishbndhn enb grnthvnnu arisikondnu knndd knd avudnnu mreabardu knd knndvnnu mreabardu
['एक वाक्य में', 'गर्म पानी का क्या हुआ?', 'गर्मी का पानी भाप बन गया।', '2 में से', 'बादल कब भारी हुआ?', 'उष्मायन वातावरण ठंडा हो गया।', 'कौन विद्यासागर से बहस करने के लिए सहमत हुआ?', 'उत्तरः -', 'तेनाली रामकृष्ण विद्यासागर के साथ बहस करने के लिए सहमत हुए', '4 टेनाली', 'रामकृष्ण ने किस ग्रंथ का चयन किया?', 'उन्होंने उतेनाली रामकृष्ण वाद के लिए तिलकस्थ महिषाबंधन नामक पुस्तक का चयन किया।', 'यह संख्या 5 है', 'हिंदी के प्रति हमें क्या याद रखना चाहिए?', 'कन्नड़ भाषा को न भूलेंः', 'जवाब दीजिएः']
